In [ ]:
##processing for the thingy uhhh semg data parsing yes yes agreed##

import csv
import pandas as pd

def find_transition_times(filename):
    with open(filename, 'r') as file:
        reader = csv.DictReader(file)
        prev_label = None
        transition_times = [None, None]
        for row in reader:
            current_label = int(row['label'])
            if prev_label is not None:
                if prev_label == 0 and current_label == 1:
                    transition_times[0] = row['time']
                elif prev_label == 1 and current_label == 2:
                    transition_times[1] = row['time']
            prev_label = current_label
        return transition_times
    
def parse_emg_data(file_in, file_out, list):
  df = pd.read_csv(file_in)
  cols_to_drop = []
  index = 0
  for name in df.columns:
    if index == 2 or index == 4 or index == 6:
      cols_to_drop.append(name)
    index = index + 1
  
  df = df.drop(columns=cols_to_drop, axis=1)
  t1 = float(list[0])
  t2 = float(list[1])

  def label_fn(val: float) -> int:
    if val < t1:
      return 0
    elif val < t2:
      return 1
    else:
      return 2
  
  df['Label'] = df.iloc[:, 0].apply(label_fn)

  return df.to_csv(file_out, index=False)
    

In [2]:
##run once dude trust me ts is gonna take a while.##
##all ts do is process all the semg data minimum 7 minutes##

directory = "parsed_emg"
subject_numbers = range(1,14)
trials = range(1,13)

for subject_number in subject_numbers:
    for trial in trials:
        self_perceived_path = "self_perceived_fatigue_index/" + "subject_" + str(subject_number) + "/Trial_" + str(trial) + ".csv"
        print("parsing self perceived path: " + self_perceived_path)
        labels = find_transition_times(self_perceived_path)
        print(labels)
        semg_path = "sEMG_data/subject_" + str(subject_number) + "/trial_" + str(trial) + ".csv"
        if (trial % 2 == 1):
            semg_output = "parsed_emg/subject_" + str(subject_number) + "/trial_" + str(trial) + "(right)" + ".csv"
        else:
            semg_output = "parsed_emg/subject_" + str(subject_number) + "/trial_" + str(trial) + "(left)" + ".csv"
        if (subject_number == 13 and trial == 12):
            labels = [168.41, 400]
        print("parsing semg data: " + semg_path)
        parse_emg_data(semg_path, semg_output, labels)

        


parsing self perceived path: self_perceived_fatigue_index/subject_1/Trial_1.csv
['49.1', '73.5']
parsing semg data: sEMG_data/subject_1/trial_1.csv
parsing self perceived path: self_perceived_fatigue_index/subject_1/Trial_2.csv
['52.24', '73.22']
parsing semg data: sEMG_data/subject_1/trial_2.csv
parsing self perceived path: self_perceived_fatigue_index/subject_1/Trial_3.csv
['108.08', '756.32']
parsing semg data: sEMG_data/subject_1/trial_3.csv
parsing self perceived path: self_perceived_fatigue_index/subject_1/Trial_4.csv
['194.46', '639.94097']
parsing semg data: sEMG_data/subject_1/trial_4.csv
parsing self perceived path: self_perceived_fatigue_index/subject_1/Trial_5.csv
['62.42', '120.98001']
parsing semg data: sEMG_data/subject_1/trial_5.csv
parsing self perceived path: self_perceived_fatigue_index/subject_1/Trial_6.csv
['111.48001', '165.679']
parsing semg data: sEMG_data/subject_1/trial_6.csv
parsing self perceived path: self_perceived_fatigue_index/subject_1/Trial_7.csv
['67.